# Advent of Code 2021
## [Day 18: Snailfish](https://adventofcode.com/2021/day/18)

#### Load Data

In [1]:
import json
import math
from collections import deque

In [2]:
import aocd
input_data = [json.loads(s) for s in aocd.get_data(year=2021, day=18).split('\n')]
input_data[:5]

[[[8, 8], 5],
 [[[[9, 0], 1], 4], [[3, 6], [0, 5]]],
 [[9, [0, [4, 5]]], [1, [[6, 8], 4]]],
 [[8, 7], [[[8, 5], [2, 0]], [[6, 3], [5, 0]]]],
 [[[1, 8], 2], [[[9, 1], [2, 0]], [1, [9, 4]]]]]

In [3]:
test_data = [
    [1,2],
    [[1,2],3],
    [9,[8,7]],
    [[1,9],[8,5]],
    [[[[1,2],[3,4]],[[5,6],[7,8]]],9],
    [[[9,[3,8]],[[0,9],6]],[[[3,7],[4,9]],3]],
    [[[[1,3],[5,3]],[[1,3],[8,7]]],[[[4,9],[6,9]],[[8,2],[7,3]]]]
]

### Part 1

In [4]:
class RegularNum(object):
    def __init__(self, value):
        self.value = value
        
    def __repr__(self):
        return f"{self.value}"
    
    def __add__(self, other):
        if isinstance(other, RegularNum):
            return RegularNum(self.value + other.value)
        else:
            return RegularNum(self.value + other)

    def __iadd__(self, other):
        if isinstance(other, RegularNum):
            self.value += other.value
        else:
            self.value += other
        return self

    def split(self):
        if self.value < 10:
            return self
        else:
            lhs = RegularNum(math.floor(self.value / 2))
            rhs = RegularNum(math.ceil(self.value / 2))
            return SnailNum(lhs, rhs)
        
    def magnitude(self):
        return self.value
    
r = RegularNum(12)
r += 1
r

13

In [5]:
class SnailNum(object):
    def __init__(self, lhs, rhs):
        self.lhs = lhs
        self.rhs = rhs
        
    def __repr__(self):
        return f"[{self.lhs},{self.rhs}]"
        
    def reduce(self):
        pass
    
    def explode(self):
        pass
    
    @classmethod
    def parse(cls, value):
        if isinstance(value, list):
            lhs = cls.parse(value[0])
            rhs = cls.parse(value[1])
            return cls(lhs, rhs)
        else:
            return RegularNum(value)
        
    def magnitude(self):
        return 3*self.lhs.magnitude() + 2*self.rhs.magnitude()
        
            
SnailNum.parse([9,[8,7]])

[9,[8,7]]

In [6]:
def find_explode(num, depth=0):
    """
    If any pair is nested inside four pairs, the leftmost such pair explodes.
    """
    if not isinstance(num, SnailNum):
        return None
    if depth >= 4:
        return [num]
    if isinstance(num, SnailNum):
        found = find_explode(num.lhs, depth+1) or find_explode(num.rhs, depth+1)
        if found:
            return [num]+found
    
s = SnailNum.parse([[[[[9,8],1],2],3],4])
path = find_explode(s)
path

[[[[[[9,8],1],2],3],4], [[[[9,8],1],2],3], [[[9,8],1],2], [[9,8],1], [9,8]]

In [7]:
def get_leftmost_leaf(node):
    if isinstance(node, SnailNum):
        return get_leftmost_leaf(node.lhs)
    else:
        return node
    
def find_leaf_to_right(path):
    path = list(path)
    prev = path.pop()
    while path:
        cursor = path.pop()
        if cursor.lhs is prev:
            return get_leftmost_leaf(cursor.rhs)
        prev = cursor
    
def get_rightmost_leaf(node):
    if isinstance(node, SnailNum):
        return get_rightmost_leaf(node.rhs)
    else:
        return node

def find_leaf_to_left(path):
    path = list(path)
    prev = path.pop()
    while path:
        cursor = path.pop()
        if cursor.rhs is prev:
            return get_rightmost_leaf(cursor.lhs)
        prev = cursor

find_leaf_to_right(path)

1

In [8]:
def do_explode(num):
    path_to_explosion = find_explode(num)
    if path_to_explosion:
        node_to_explode = path_to_explosion[-1]
        leftie = find_leaf_to_left(path_to_explosion)
        if leftie:
            leftie += node_to_explode.lhs
        rightie = find_leaf_to_right(path_to_explosion)
        if rightie:
            rightie += node_to_explode.rhs
        parent = path_to_explosion[-2]
        if parent.lhs is node_to_explode:
            parent.lhs = RegularNum(0)
        else:
            parent.rhs = RegularNum(0)
        return True
    return False
            
s = SnailNum.parse([[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]])
do_explode(s)
s

[[3,[2,[8,0]]],[9,[5,[7,0]]]]

In [9]:
def find_split(num):
    """
    If any regular number is 10 or greater, the leftmost such regular number splits.
    """
    if isinstance(num, RegularNum) and num.value >= 10:
        return [num]
    if isinstance(num, SnailNum):
        found = find_split(num.lhs) or find_split(num.rhs)
        if found:
            return [num]+found
    
s = SnailNum.parse([[[[0,7],4],[15,[0,13]]],[1,1]])
find_split(s)

[[[[[0,7],4],[15,[0,13]]],[1,1]], [[[0,7],4],[15,[0,13]]], [15,[0,13]], 15]

In [10]:
def do_split(num):
    """Modify num with a split operation if possible"""
    path_to_split = find_split(num)
    if path_to_split:
        node_to_split = path_to_split[-1]
        parent = path_to_split[-2]
        if parent.lhs is node_to_split:
            parent.lhs = node_to_split.split()
        else:
            parent.rhs = node_to_split.split()
        return True
    return False
            
s = SnailNum.parse([[[[0,7],4],[15,[0,13]]],[1,1]])
do_split(s)
s

[[[[0,7],4],[[7,8],[0,13]]],[1,1]]

In [11]:
def reduce(num):
    any_action = True
    while any_action:
        any_action = do_explode(num) or do_split(num)
    return num

s = SnailNum.parse([[[[[4,3],4],4],[7,[[8,4],9]]],[1,1]])
reduce(s)

[[[[0,7],4],[[7,8],[6,0]]],[8,1]]

In [12]:
def addition(n1, n2):
    result = SnailNum(n1, n2)
    reduce(result)
    return result

s1 = SnailNum.parse([[[[4,3],4],4],[7,[[8,4],9]]])
s2 = SnailNum.parse([1,1])
addition(s1, s2)

[[[[0,7],4],[[7,8],[6,0]]],[8,1]]

In [13]:
test_data = [
    [[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]],
    [[[5,[2,8]],4],[5,[[9,9],0]]],
    [6,[[[6,2],[5,6]],[[7,6],[4,7]]]],
    [[[6,[0,7]],[0,9]],[4,[9,[9,0]]]],
    [[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]],
    [[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]],
    [[[[5,4],[7,7]],8],[[8,3],8]],
    [[9,3],[[9,9],[6,[4,9]]]],
    [[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]],
    [[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]
]

In [14]:
def homework(input_data):
    result = SnailNum.parse(input_data[0])
    for n in input_data[1:]:
        num = SnailNum.parse(n)
        result = addition(result, num)
    return result
homework(test_data).magnitude()

4140

#### Part 1 Answer
Add up all of the snailfish numbers from the homework assignment in the order they appear.  
**What is the magnitude of the final sum?**

In [15]:
homework(input_data).magnitude()

3725

### Part 2

In [16]:
def largest_mag(input_data):
    largest_seen = 0
    for i in range(len(input_data)):
        for j in range(len(input_data)):
            if i == j:
                continue
            num_i = SnailNum.parse(input_data[i])
            num_j = SnailNum.parse(input_data[j])
            mag = addition(num_i, num_j).magnitude()
            if mag > largest_seen:
                largest_seen = mag
    return largest_seen
largest_mag(test_data)

3993

#### Part 2 Answer
**What is the largest magnitude of any sum of two different snailfish numbers from the homework assignment?**

In [17]:
largest_mag(input_data)

4832